In [48]:
import os
import sys

root = os.path.dirname(os.path.abspath("."))
sys.path.append(root)

import numpy as np
import pandas as pd

import predikit as pk

# Test Load DataFrame


In [49]:
from io import BytesIO
from pathlib import Path


def load_show(path_or_buf, extension=None, n=3, label=None, **kwargs):
    df = pk.DataFrameParser(
        path_or_buf=path_or_buf,
        extension=extension,
        **kwargs,
    )
    if label:
        display(f"From {label}")

    display(df.get_column_names())
    display(df.get_column_types())
    display(df.get_column_types(parsed=True))
    display(df.get_numeric_columns())
    display(df.get_non_numeric_columns())
    display(df.head(n))


# from a Buffered Input Stream
f = BytesIO(b"a,b,c\n1,2,3\n4,5,6\n7,8,9")
load_show(f, extension="csv", label="BytesIO")

# from a csv file
f = Path("./sample_data/airline_bumping.csv")
print(f)
load_show(f, n=4, label="csv")

# from a pickle file
f = Path("./sample_data/stations.pickle")
load_show(f, n=5, label="pickle")

# from a parquet file
f = Path("./sample_data/Flights 1m.parquet")
load_show(f, n=3, label="parquet")

# from a dictionary
data = {
    "Name": ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
    "Age": [np.nan, 23, 25, 50, np.nan, 50],
    "Credit": [np.nan, 400, np.nan, 50, 200, np.nan],
}
load_show(data, n=3, label="dict")

# from a list of dictionaries
data = [
    {"Name": "John", "Age": np.nan, "Credit": np.nan},
    {"Name": "Andrea", "Age": 23, "Credit": 400},
    {"Name": "Rose", "Age": 25, "Credit": np.nan},
    {"Name": "Linda", "Age": 50, "Credit": 50},
    {"Name": "Peter", "Age": np.nan, "Credit": 200},
    {"Name": "Meg", "Age": 50, "Credit": np.nan},
]
load_show(data, label="list of dicts")

# from a dictionary of Series
data = {
    "one": pd.Series([1.0, 2.0, 3.0], index=["a", "b", "c"]),
    "two": pd.Series([1.0, 2.0, 3.0, 4.0], index=["a", "b", "c", "d"]),
}
load_show(data, label="dict of Series")

# from a 2d array
data = np.array(
    [
        ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
        [np.nan, 23, 25, 50, np.nan, 50],
        [np.nan, 400, np.nan, 50, 200, np.nan],
    ]
)

data = data.T
load_show(data, label="2d array", columns=["Name", "Age", "Credit"])


# ToDo => add tests for JSON and EXCEL

'From BytesIO'

['a', 'b', 'c']

{'a': dtype('int64'), 'b': dtype('int64'), 'c': dtype('int64')}

{'a': 'i', 'b': 'i', 'c': 'i'}

['a', 'b', 'c']

...

{'Name': dtype('O'), 'Age': dtype('O'), 'Credit': dtype('O')}

{'Name': 'O', 'Age': 'O', 'Credit': 'O'}

None

['Name', 'Age', 'Credit']

,Name,Age,Credit
0,John,nan,nan
1,Andrea,23,400
2,Rose,25,nan


# Helper Functions


In [50]:
from result import Result


def unwrap_value_or_error(result: Result):
    if result.is_ok():
        return result.unwrap()
    return result.unwrap_err()


def init_df_sample(sample_number: int = 1):
    if sample_number < 0 or sample_number > 4:
        raise ValueError("sample_number must be between 0 and 4")

    data = {
        "Name": ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
        "Age": [np.nan, 200, 25, 50, np.nan, 50],
        "Credit": [np.nan, 400, np.nan, 200_00_00, 1_000_000, np.nan],
    }
    samples = {
        1: data,
        2: "./sample_data/airline_bumping.csv",
        3: "./sample_data/Flights 1m.parquet",
        4: "./sample_data/stations.pickle",
    }

    return pk.DataFrameParser(path_or_buf=samples[sample_number])

# Data Cleansing


## Handling Missing Values


In [51]:
df = init_df_sample()
display(df)
# bug in MODE doesn't fill all NaNs
mvp = pk.MissingValuesProcessor(strategy=pk.MissingValueStrategy.MEDIAN)
result = mvp.fit_transform(df)

if result.is_err():
    raise ValueError(
        "Operation must be done for later operation (Outliers Detection)"
    )

df = result.unwrap()

,Name,Age,Credit
0,John,NaN,NaN
1,Andrea,200.0,400.0
2,Rose,25.0,NaN
3,Linda,50.0,2000000.0
4,Peter,NaN,1000000.0
5,Meg,50.0,NaN


## Handling Outliers


In [52]:
display(df)
op = pk.OutliersProcessor(
    pk.OutlierDetectionMethod.Z_SCORE,
    threshold=3,
    verbose=True,
    add_indicator=True,
)
df = op.fit_transform(df, columns=["Age"]).unwrap()

df

,Name,Age,Credit
0,John,50.0,1000000.0
1,Andrea,200.0,400.0
2,Rose,25.0,1000000.0
3,Linda,50.0,2000000.0
4,Peter,50.0,1000000.0
5,Meg,50.0,1000000.0


2023-12-18 01:46:53,363 (data_cleansing.py:606) - INFO: Number of outliers detected: 2 in Feature Age
2023-12-18 01:46:53,366 (data_cleansing.py:610) - INFO: Proportion of outlier detected: 33.3%


,Name,Age,Credit,Age_isOutlier_Z_SCORE
0,John,50.0,1000000.0,0
1,Andrea,50.0,400.0,1
2,Rose,50.0,1000000.0,1
3,Linda,50.0,2000000.0,0
4,Peter,50.0,1000000.0,0
5,Meg,50.0,1000000.0,0


# Encoding Features


In [53]:
enc = pk.EncodingProcessor(
    pk.EncodingStrategies.OneHotEncoder,
    verbose=True,
    sparse_output=True,
)
cols = ["Name"]
enc.fit_transform(df, columns=cols).unwrap()

,Age,Credit,Age_isOutlier_Z_SCORE,Name_Andrea,Name_John,Name_Linda,Name_Meg,Name_Peter,Name_Rose
0,50.0,1000000.0,0,0.0,1.0,0.0,0.0,0.0,0.0
1,50.0,400.0,1,1.0,0.0,0.0,0.0,0.0,0.0
2,50.0,1000000.0,1,0.0,0.0,0.0,0.0,0.0,1.0
3,50.0,2000000.0,0,0.0,0.0,1.0,0.0,0.0,0.0
4,50.0,1000000.0,0,0.0,0.0,0.0,0.0,1.0,0.0
5,50.0,1000000.0,0,0.0,0.0,0.0,1.0,0.0,0.0


# Filtering Data


In [61]:
df = init_df_sample(2)
col: str = "year"
# display(df)
bf = pk.BasicFilteringProcessor(
    "in",
    case_sensitive=False,
    verbose=True,
)

# bf
res = bf.fit_transform(df, column=None)

unwrap_value_or_error(res)
# result_unpacked = result.unwrap()
# print(result_unpacked)
# bf
# display(df)


# bf.set_params(operator=pk.FilterOperator.EQUAL, value=50)
# display(bf.fit_transform(df, column="Age"))

'Column name must be provided'

# String Modifications


In [55]:
data = {
    "Name": [
        "   John   .!,@  ",
        "#A nd@rea$",
        "Rose",
        "L0ind9a",
        "P!e1t@e_r",
        "M+e - g$",
    ],
    "Age": [np.nan, 200, 25, 50, np.nan, 50],
    "Credit": [np.nan, 400, np.nan, 200_00_00, 1_000_000, np.nan],
}
df = pk.DataFrameParser(data)

sop = pk.StringOperationsProcessor(
    "title",
    trim=True,
    remove_letters=False,
    remove_whitespace=True,
    remove_numbers=True,
    remove_punctuation=True,
    verbose=False,
)


display(df)
result = sop.fit_transform(df, columns=["Name"])
unwrap_value_or_error(result)

,Name,Age,Credit
0,"John .!,@",NaN,NaN
1,#A nd@rea$,200.0,400.0
2,Rose,25.0,NaN
3,L0ind9a,50.0,2000000.0
4,P!e1t@e_r,NaN,1000000.0
5,M+e - g$,50.0,NaN


,Name,Age,Credit
0,John,NaN,NaN
1,Andrea,200.0,400.0
2,Rose,25.0,NaN
3,Linda,50.0,2000000.0
4,Peter,NaN,1000000.0
5,Meg,50.0,NaN


# Data Cleanse as a Whole


In [56]:
data = {
    "Name": [
        "   John   .!,@  ",
        "#A nd@rea$",
        "Rose",
        "L0ind9a",
        "P!e1t@e_r",
        "M+e - g$",
    ],
    "Age": [np.nan, 200, 25, 50, np.nan, 50],
    "Credit": [np.nan, 400, np.nan, 200_00_00, 1_000_000, np.nan],
}

dc = pk.DataCleanser(
    missing_clean=True,
    missing_strategy="median",
    missing_indicator=True,
    missing_fill_value=0,
    outlier_clean=True,
    outlier_method="z",
    outlier_threshold=1.5,
    outlier_indicator=True,
    str_trim=True,
    str_remove_whitespace=True,
    str_remove_letters=True,
    str_remove_numbers=True,
    str_remove_punctuation=True,
    verbose=False,
)

result = dc.fit_transform(df, columns=["Name"])
unwrap_value_or_error(result)

2023-12-18 01:46:53,534 (data_cleansing.py:972) - INFO: > Cleansing


'Selected columns are of non-numeric type. Unable to process missing values on non-numeric columns. When using median strategy, only numeric columns are allowed.'

# Output Dataset


In [57]:
import platform

print(f"Running on {platform.system()}")

# cross platform
dfe = pk.DataFrameExporter(df, extension=pk.FileExtension.CSV)
dfe.export()

2023-12-18 01:46:53,566 (output.py:87) - INFO: Exporting to /home/mghali/predikit_out/out.csv ...


Running on Linux


In [58]:
from predikit.preprocessing import FeatureSelection

data = {
    "Name": ["John", "Andrea", "Rose", "Linda", "Peter", "Meg"],
    "Age": [21, 23, 25, 50, 48, 50],
    "Credit": [74512, 400, 56132, 50, 1_000_000, 45121],
}

df = pk.DataFrameParser(data)
fs = FeatureSelection(exclude_dtypes=["object"], verbose=True)
# new_df = fs.fit_transform(df, columns=["Age"], dtypes=["object"])
# new_df = fs.fit_transform(df)
# new_df = fs.fit_transform(df, dtypes=["object"])
# new_df = fs.fit_transform(df, columns=["Age"])
# new_df = fs.fit_transform(df, columns=["Name"], dtypes=["number"])
result = fs.fit_transform(df, columns=["Credit", "Age"])
unwrap_value_or_error(result)

'This results in an empty data frame'